In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
import optuna
import os
import numpy as np

## load data

In [2]:
country = 'iran'
mode = 'Train'
feature = 'indices'

path = '/app/stella/dev/GeoITU/data'

X_indices = np.load(os.path.join(path, '{}_{}_{}.npy'.format(country, mode, feature)), allow_pickle=True)
y_labels = np.load(os.path.join(path, '{}_{}_labels.npy'.format(country, mode)), allow_pickle=True)

            
print(X_indices.shape, y_labels.shape)
assert X_indices.shape[0] == y_labels.shape[0]

X_indices = X_indices.reshape(X_indices.shape[0], -1)

(500, 55, 15) (500,)


## rf

In [7]:
# Setting up the stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []  # List to store accuracy of each fold
f1_scores = []
fold_num = 0
num_trial = 50

# 5-fold Stratified Cross Validation loop
for train_index, test_index in skf.split(X_indices, y_labels):
    fold_num += 1

    # Splitting the dataset for this fold
    X_train, X_test = [X_indices[i] for i in train_index], [X_indices[i] for i in test_index]
    y_train_labels, y_test_labels = [y_labels[i] for i in train_index], [
        y_labels[i] for i in test_index
    ]
    
    
    # optimize params for single fold
    if fold_num == 1:  


        def objective(trial):
            params = {
            'n_estimators' : trial.suggest_int('n_estimators', 100, 1000),
            'max_depth' : trial.suggest_int('max_depth', 3, 15),
            # 'max_features' : trial.suggest_categorical('max_features', ['auto', 'sqrt']), 
            'min_samples_split' : trial.suggest_int('min_samples_split', 5, 32),
            'bootstrap' : trial.suggest_categorical('bootstrap', [True, False]),
            'n_jobs' : trial.suggest_categorical('n_jobs', [-1]) #fixed. use all cpus
            }

            clf = RandomForestClassifier(**params)
            clf.fit(X_train, y_train_labels)

            # Making predictions on the test set
            y_pred = clf.predict(X_test)

            # Calculating and reporting the accuracy
            accuracy = accuracy_score(y_test_labels, y_pred)
            return accuracy


        # optimize study
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=num_trial) 
        print(study.best_params)
                        
    # Model training
    clf = RandomForestClassifier(**study.best_params)  # change classifier here
    clf.fit(X_train, y_train_labels)
    

    # Making predictions on the test set
    y_pred = clf.predict(X_test)

    # Calculating and reporting the accuracy
    accuracy = accuracy_score(y_test_labels, y_pred)
    accuracies.append(accuracy)  # Storing the accuracy
    
    # Calculating and reporting the fscore
    f_score = f1_score(y_test_labels, y_pred, average='weighted')
    f_scores.append(f_score)  # Storing the accuracy
    print(f"Fold {fold_num} Accuracy: {accuracy}")
    print(f"Fold {fold_num} F_score: {f_score}")

# Reporting the final results
avg_accuracy = np.mean(accuracies)
avg_fscore = np.mean(f_scores)
print(f"Average Accuracy across all folds: {avg_accuracy:.4f}")
print(f"Average Fscore across all folds: {avg_fscore:.4f}")


[I 2023-10-03 17:28:26,726] A new study created in memory with name: no-name-5f616e2e-5cdd-472c-8326-c79783130b83
[I 2023-10-03 17:28:27,793] Trial 0 finished with value: 0.97 and parameters: {'n_estimators': 352, 'max_depth': 8, 'min_samples_split': 19, 'bootstrap': False, 'n_jobs': -1}. Best is trial 0 with value: 0.97.
[I 2023-10-03 17:28:29,053] Trial 1 finished with value: 0.95 and parameters: {'n_estimators': 489, 'max_depth': 7, 'min_samples_split': 31, 'bootstrap': False, 'n_jobs': -1}. Best is trial 0 with value: 0.97.
[I 2023-10-03 17:28:31,299] Trial 2 finished with value: 0.96 and parameters: {'n_estimators': 916, 'max_depth': 4, 'min_samples_split': 14, 'bootstrap': False, 'n_jobs': -1}. Best is trial 0 with value: 0.97.
[I 2023-10-03 17:28:32,088] Trial 3 finished with value: 0.96 and parameters: {'n_estimators': 251, 'max_depth': 10, 'min_samples_split': 14, 'bootstrap': True, 'n_jobs': -1}. Best is trial 0 with value: 0.97.
[I 2023-10-03 17:28:33,614] Trial 4 finished w

{'n_estimators': 352, 'max_depth': 8, 'min_samples_split': 19, 'bootstrap': False, 'n_jobs': -1}
Fold 1 Accuracy: 0.96
Fold 1 F_score: 0.959983993597439
Fold 2 Accuracy: 0.95
Fold 2 F_score: 0.949874686716792
Fold 3 Accuracy: 0.97
Fold 3 F_score: 0.9699729756781104
Fold 4 Accuracy: 0.97
Fold 4 F_score: 0.9699729756781104
Fold 5 Accuracy: 0.96
Fold 5 F_score: 0.96
Average Accuracy across all folds: 0.9620
Average Fscore across all folds: 0.9613
